In [339]:
%load_ext autoreload
%autoreload 2

import __confing
__confing.change_to_root_folder()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Current directory: c:\Users\Leo\OneDrive\Escritorio\Final Proyect


In [399]:
import pandas as pd
import numpy as np

from src import value_filler as vs
from src import feature_selection as fs
from src import data_procces as dp

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from src import evaluation

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report, mean_squared_error,roc_auc_score,confusion_matrix
from sklearn.model_selection import KFold

In [400]:
def feature_importance(model, df:pd.DataFrame, target:str, n:int):
    importances = model.feature_importances_
    indices = np.argsort(importances)[::-1]
    
    feature_names_original = list(df.columns)
    feature_names_original.remove(target)
    
    max_importance = list(indices[:n])
    
    nueva_lista = [feature_names_original[i] for i in max_importance]
    
    return nueva_lista

In [401]:
def get_performance(model, X_train, X_test, y_train, y_test):
    y_pred = model.predict(X_test)
    roc_auc_train = roc_auc_score(y_train, model.predict(X_train))
    # Validation ROC AUC Score
    acc = accuracy_score(y_test, y_pred)
    roc_auc_val = roc_auc_score(y_test, y_pred)
    print( f"Model Accuracy: {acc}",
            f"Train roc_auc: {roc_auc_train}", 
            f"Test roc_auc: {roc_auc_val}",
            sep='\n', end='\n\n')
    print(confusion_matrix(y_test, y_pred),
    classification_report(y_test, y_pred),
    sep='\n'
)

In [446]:
n=3
df=pd.read_csv(f'data/waves/wave{n}_raw.csv')
target=f'r{n}hosp1y'
drop=[f'r{n}hspnit1y'] # si la wave3 falla quitar estas columnas r3bpsft r3bpref

In [447]:
# Drope
df = df.drop(drop, axis=1)

df = df.dropna(subset=[target]) 

porcentaje_nulos = df.isnull().sum(axis=1) / len(df.columns)
df = df[(df[target] != 0) | (porcentaje_nulos <= 0.4)]

In [448]:
# split
y=df[target]
X=df.drop(target,axis=1)
X = fs.fast_fill(X)
# X=X[columns_selected]
X_train, X_test, y_train, y_test=dp.split_data(X, y, test_size=0.2, random_state=40)

In [449]:
# resample 
X_train, y_train = dp.apply_resample(X_train, y_train,v=1.35)
X_test, y_test = dp.apply_resample(X_test, y_test,v=1)

In [450]:
# smote
X_train, y_train = dp.apply_smote(X_train, y_train)

In [451]:
# scaler
X_train, X_test = dp.apply_standard_scaler(X_train, X_test)

In [452]:
# # power transform
X_train, X_test = dp.apply_power_transform(X_train, X_test)

In [453]:
import xgboost as xgb
from sklearn.model_selection import KFold

def train_xgboost_with_kfold(features, labels, model):
    # Definir el número de folds (k)
    k = 7
    
    kf = KFold(n_splits=k)
    
    # Iterar sobre los folds
    for train_index, test_index in kf.split(features):
        # Dividir los datos en conjuntos de entrenamiento y prueba
        X_train, X_test = features[train_index], features[test_index]
        y_train, y_test = labels[train_index], labels[test_index]
        
        # Entrenar el modelo XGBoost
        model.fit(X_train, y_train)
    
    return model

model_untrained = xgb.XGBClassifier()
model= train_xgboost_with_kfold(X_train, y_train,model_untrained)

model_2 = xgb.XGBClassifier()
model_2.fit(X_train, y_train)

pass

In [454]:
get_performance(model, X_train, X_test, y_train, y_test)
print('######################################################')
print('')
get_performance(model_2, X_train, X_test, y_train, y_test)

Model Accuracy: 0.9029850746268657
Train roc_auc: 0.9974515800203874
Test roc_auc: 0.9029850746268656

[[318  17]
 [ 48 287]]
              precision    recall  f1-score   support

         0.0       0.87      0.95      0.91       335
         1.0       0.94      0.86      0.90       335

    accuracy                           0.90       670
   macro avg       0.91      0.90      0.90       670
weighted avg       0.91      0.90      0.90       670

######################################################

Model Accuracy: 0.9059701492537313
Train roc_auc: 1.0
Test roc_auc: 0.9059701492537313

[[323  12]
 [ 51 284]]
              precision    recall  f1-score   support

         0.0       0.86      0.96      0.91       335
         1.0       0.96      0.85      0.90       335

    accuracy                           0.91       670
   macro avg       0.91      0.91      0.91       670
weighted avg       0.91      0.91      0.91       670



In [455]:
print(feature_importance(model, df, target, n=50))

['r3rfcaredpmm', 'r3gripcomp', 'r3lgmusam', 'r3cesdm_m', 'r3oophos1y', 'r3rscarehrm', 'r3fvscan', 'r3hrtatlmt', 'r3domhand', 'r3nopencil', 'r3gpcare', 'r3wspeed1', 'r3shophlp', 'r3rxdiabi', 'r3walkaid_m', 'r3walkcomp', 'r3mobilsevm', 'r3balrtcomp', 'r3bathehlp', 'r3fatigue', 'r3rgrip2', 'r3nagi10', 'r3rafaany', 'r3oopfhho1y', 'r3doctim1y', 'r3cholst', 'r3mobila', 'r3jcten', 'r3retage', 'r3grossa', 'r3henum', 'r3rxarthr', 'r3breath_m', 'r3hltc', 'r3bed', 'r3wspeed', 'r3stoopa', 'r3clim1a', 'r3racaany', 'r3stoop', 'r3fallinj', 'r3fbwc20_m', 'r3hip_m', 'r3rfcare', 'r3money', 'r3rascaredpm', 'r3socwk', 'r3cage', 'r3doctor1y', 'r3jlocc_m']


In [456]:
print(feature_importance(model_2, df, target, n=50))

['r3gripcomp', 'r3rfcaredpmm', 'r3proxy', 'r3rscarehrm', 'r3oophos1y', 'r3hrtatlmt', 'r3stoopa', 'r3fineam', 'r3cholst', 'r3balrtcomp', 'r3fatigue', 'r3nagi8m', 'r3nagi10', 'r3rxdiabi', 'r3rafcare', 'r3bathehlp', 'r3retage', 'r3shophlp', 'r3walkaid_m', 'r3grossa', 'r3doctim1y', 'r3rfaany', 'r3rccare', 'r3bedhlp', 'r3walkcomp', 'r3sustdfe_m', 'r3mobila', 'r3enlife', 'r3rfcare', 'r3rxdiab', 'r3retemp', 'r3papsm', 'r3arms', 'r3domhand', 'r3shlt', 'r3nopencil', 'r3henum', 'r3rifcaredpm', 'r3fopenup_m', 'r3hip_m', 'r3raccare', 'r3jlasty', 'r3jlocc_m', 'r3riccare', 'r3rgrip2', 'r3covs_m', 'r3hrtatte', 'r3diabe', 'r3novisual', 'r3toilt']


Model Accuracy: 0.9029850746268657
Train roc_auc: 0.9982161060142712
Test roc_auc: 0.9029850746268656

Model Accuracy: 0.9029850746268657
Train roc_auc: 1.0
Test roc_auc: 0.9029850746268656